# Proceso **ETL** orientado a un modelo de Análisis de Sentimiento (Portugués)

## 1-. El Dataset




El **DataSet** utilizado para el proceso es un conjunto de datos (*sentiment_reviews_olist_pt.csv*) de resultados del análisis de sentimiento aplicado a las reseñas de clientes del e-commerce brasileño Olist, disponible en la plataforma [Kaggle](https://www.kaggle.com/), para tareas de procesamiento de lenguaje natural como clasificación, generación de texto o análisis de sentimiento, disponible en:

[Olist – Sentiment Analysis - Kaggle](https://www.kaggle.com/datasets/vanesamizrahi/olist-sentiment-analysis-huggingface?resource=download)

## 2-. Contenido y estructura del dataset


### 2.1-. El dataset incluye:

1.  Reseñas en portugués

    Cada registro contiene texto de reseña en Portugués.

2.  Campos principales por registro

    Los atributos mas relevantes para nuestro aplicación que contiene cada fila son:
    - `review_id`: identificador único de la reseña, utilizado para la trazabilidad y referencia de cada comentario dentro del conjunto de datos.
    - `order_id`: identificador del pedido asociado a la reseña, que permite relacionar el comentario con información transaccional o logística.
    - `review_comment_message`: texto libre escrito por el cliente, que expresa su opinión sobre el producto o servicio. Este campo constituye la entrada principal para el análisis de sentimientos mediante técnicas de procesamiento de lenguaje natural.
    - `review_score`: calificación numérica otorgada por el usuario en una escala de 1 a 5, considerada como la etiqueta original de referencia (ground truth) para la generación de las clases de sentimiento.
    - `sentiment`: etiqueta categórica derivada de la calificación numérica, donde los valores 1–2 se asignan a NEGATIVO, 3 a NEUTRO y 4–5 a POSITIVO. Este atributo se utiliza como variable objetivo en los modelos de clasificación de sentimientos.
Para nuestro caso, nos enfocaremos en las columnas `"review_comment_message"` y `"sentiment_stars"`.

## 3-. Librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 4-. Extracción de archivos (Extract)
En esta fase obtenemos los datos desde su fuente original. Se cargar la información sin modificarla, preservando la evidencia original.

In [3]:
df = pd.read_csv('sentiment_reviews_olist_pt.csv')

In [4]:
df.head(3)

,review_id,order_id,review_score,review_comment_message,sentiment_label_raw,sentiment_score_raw,sentiment_stars,sentiment_polarity
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.,3 stars,0.399876,3,0
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...,5 stars,0.538074,5,1
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,aparelho eficiente. no site a marca do aparelh...,3 stars,0.361089,3,0


### 4.1-. Validaciones iniciales

Aqui vemos las columnas que existen, texto del comentario, las etiquetas y el tamaño del DataSet.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40950 entries, 0 to 40949
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   review_id               40950 non-null  object 
 1   order_id                40950 non-null  object 
 2   review_score            40950 non-null  int64  
 3   review_comment_message  40950 non-null  object 
 4   sentiment_label_raw     40950 non-null  object 
 5   sentiment_score_raw     40950 non-null  float64
 6   sentiment_stars         40950 non-null  int64  
 7   sentiment_polarity      40950 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 2.5+ MB


In [6]:
df.shape

(40950, 8)

In [7]:
df['sentiment_stars'].describe()

,sentiment_stars
count,40950.000000
mean,3.256361
std,1.746147
min,1.000000
25%,1.000000
50%,4.000000
75%,5.000000
max,5.000000


## 5-. Trasformación

Aquí se limpia, normaliza y estructura el texto para que el modelo pueda aprender patrones reales.

La transformación se divide en sub-etapas lógicas.

1.   Selección de columnas relevantes
2.   Eliminación de nulos
3.   Normalización de idioma
4.   Eliminación de comentarios duplicados
5.   Limpieza de texto (Text Cleaning)
6.   Creación de la etiqueta de sentimiento (Labeling)
7.   Balanceo y validación de clases
8.   Análisis estadístico para tomar un subconjunto de datos del DataSet (5000), para el entrenamiento del modelo.



In [8]:
df.head(3)

,review_id,order_id,review_score,review_comment_message,sentiment_label_raw,sentiment_score_raw,sentiment_stars,sentiment_polarity
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.,3 stars,0.399876,3,0
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...,5 stars,0.538074,5,1
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,aparelho eficiente. no site a marca do aparelh...,3 stars,0.361089,3,0


### 5.1-. Selección de columnas relevantes

Eliminar ruido mejora rendimiento y reduce sesgo.

In [9]:
df = df[['review_comment_message', 'sentiment_stars']]

### 5.2-. Eliminación de elementos nulos

Un comentario vacío no aporta información semántica.
Es un método de pandas que sirve para eliminar valores nulos (`NaN, None, NaT`) del DataFrame.

Quá hace por defecto:

- Elimina filas.

- Elimina la fila completa si encuentra al menos un valor nulo (esto se puede controlar con parámetros).

`subset=['review_comment_message']` Este parámetro indica en qué columna(s) se debe verificar la existencia de valores nulos.

En este caso:

- Solo se revisa la columna review_body.

- Si review_body es NaN en una fila, esa fila se elimina.

- Los valores nulos en otras columnas no afectan la eliminación.

Es necesario por que los valores nulos en columnas de texto:

- Provocan errores al tokenizar

- Dañan el entrenamiento del modelo

- Generan resultados inconsistentes

In [10]:
df = df.dropna(subset=['review_comment_message'])

### 5.4-. Eliminación de comentarios duplicados

Los comentarios duplicados sesgan el modelo, haciéndolo aprender frases específicas en lugar de patrones generales.

- `df['review_comment_message'].str.lower()` ->  Convierte todo el texto de la columna a minúsculas.
- `df['review_comment_message'].str.lower().str.strip()` -> Elimina espacios en blanco al inicio y al final del texto.

El texto ya normalizado se guarda en una nueva columna `'review_body_clean'`, preservando el texto original sin modificar.

- Permite trazabilidad

- Facilita auditoría

- Evita pérdida de información

`df.drop_duplicates(...)` -> Método de pandas para eliminar filas duplicadas.

`subset=['review_comment_message_clean']` -> Indica que la comparación de duplicados se hace solo sobre la columna `review_comment_message_clean`.


In [11]:
df['review_clean'] = (
    df['review_comment_message']
    .str.lower()
    .str.strip()
)
df = df.drop_duplicates(subset=['review_clean'])

In [12]:
df.head(3)

,review_comment_message,sentiment_stars,review_clean
0,Recebi bem antes do prazo estipulado.,3,recebi bem antes do prazo estipulado.
1,Parabéns lojas lannister adorei comprar pela I...,5,parabéns lojas lannister adorei comprar pela i...
2,aparelho eficiente. no site a marca do aparelh...,3,aparelho eficiente. no site a marca do aparelh...


### 5.5-. Limpieza de texto (Text Cleaning)

El texto crudo puede contiener: mayúsculas, símbolos, números saltos de línea, etc. Todo esto debe estandarizarce.

También debemos remover la puntuación en los comentarios. Normalmente se eliminan:
- .,;:!?¿¡
- Comillas
- Paréntesis
- Símbolos especiales

`df['review_clean'].str.lower()` -> Convierte a minusculas todo los comentarios.

`.str.replace(r'http\S+', '', regex=True)` -> Elimina URLs.

`.str.replace(r'[^a-záéíóúñü\s]', '', regex=True)` ->
- ^ → negar, no reemplazar ^a-záéíóúñü\s → solo admite letras en español.

`.str.replace(r'\s+', ' ', regex=True)` -> Elimina espacios multiples.

In [13]:
df['review_clean'] = (
    df['review_clean']
    .str.lower()
    .str.replace(r'http\S+', '', regex=True)
    .str.replace(r'[^a-záéíóúñü\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

df = df.drop_duplicates(subset=['review_clean'])

### 5.6-. Creación de la etiqueta de sentimiento (Labeling)

Labeling es el proceso de asignar una clase objetivo (sentimiento) a cada observación, basándose en una fuente confiable, para permitir el entrenamiento de modelos supervisados.

Ejemplo usando estrellas:

- 1-2 → Negativo
- 3 → Neutro
- 4–5 → Positivo

In [14]:
def sentimiento(stars):
    if stars <= 2:
        return "NEGATIVO"
    elif stars == 3:
        return "NEUTRO"
    else:
        return "POSITIVO"

df['sentiment'] = df['sentiment_stars'].apply(sentimiento)

In [15]:
import re

all_words = (
    df["review_clean"]
    .astype(str)
    .str.lower()
    .str.replace(r"[^\wáéíóúãõçêôí]", " ", regex=True)
    .str.split()
    .explode()
)


In [16]:
word_lengths = all_words.str.len()


In [17]:
max_len = word_lengths.max()
longest_words = all_words[word_lengths == max_len].unique()

max_len, longest_words

(200.0,
 array(['bommmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm'],
       dtype=object))

In [18]:
df[
    df["review_clean"].str.contains(longest_words[0], regex=False)
]["review_clean"].head()

,review_clean
34945,bommmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm...


Letras repetidas

In [19]:
def normalize_repeated_letters(text, max_repeats=2):
    text = str(text).lower()
    pattern = rf"(.)\1{{{max_repeats},}}"
    return re.sub(pattern, r"\1" * max_repeats, text)

In [20]:
df["review_clean"] = df["review_clean"].apply(normalize_repeated_letters)

In [21]:
all_words = (
    df["review_clean"]
    .str.split()
    .explode()
)

max_len = all_words.str.len().max()
longest_words = all_words[all_words.str.len() == max_len].unique()

max_len, longest_words

(194.0,
 array(['msnsksknsjsjsjsnsjsjsjsnsjsjsndnndjxjxndndjsjdjdjdjdjjdjdjdjsjsjdjdjdjdjdjsjdjdjjdjdjdjdjdjsjsjdjsksjsjjssnjsjsjsjsjsjsjsjjskskskskskskkskskskskskskskskskksksksnwksksksksnnsjsjsjsjsjnsnsjsjsjsns'],
       dtype=object))

Separar por puntuacion

In [22]:
def normalize_punctuation(text):
    text = str(text).lower()
    # Reemplaza . , ; : ! ? por espacio
    text = re.sub(r"[.,;:!?]+", " ", text)
    return text

In [23]:
df["review_clean"] = df["review_clean"].apply(normalize_punctuation)

Eliminar comentarios que contengan al menos una palabra con más de 20 caracteres

In [24]:
def has_long_word(text, max_len=20):
    # Normalizamos a string y limpiamos signos
    words = re.findall(r"\b\w+\b", str(text))
    return any(len(word) > max_len for word in words)

# Crear una máscara booleana
mask_long_words = df["review_clean"].apply(has_long_word)

# Filtrar: nos quedamos SOLO con los comentarios válidos
df_clean = df[~mask_long_words].reset_index(drop=True)

In [25]:
print(f"Comentarios originales: {len(df)}")
print(f"Comentarios eliminados: {mask_long_words.sum()}")
print(f"Comentarios finales: {len(df_clean)}")

Comentarios originales: 33684
Comentarios eliminados: 46
Comentarios finales: 33638


In [26]:
df[mask_long_words]["review_clean"].head(3)


,review_clean
2857,produto foi entregue bem antes da data marcada...
4388,excelente obrigado chegou antes do prazo tudo ...
6296,todos deviam comprar aqui no baratheonameipara...


### 5.7-. Balanceo y validación de clases

La validación de clases consiste en verificar la distribución de tu variable objetivo (sentiment) antes de entrenar el modelo.

Objetivo:

- Confirmar que todas las clases existen

- Ver si alguna clase domina a las demás

- Detectar posibles sesgos

Sin esta validación, puedes entrenar un modelo aparentemente bueno, pero inútil en la práctica.

Un dataset desbalanceado provoca modelos sesgados.

In [27]:
df = df.dropna(subset=['review_clean'])

In [28]:
df = df.dropna(subset=['review_comment_message'])

In [29]:
df = df.dropna(subset=['sentiment_stars'])

In [30]:
df.head(2)

,review_comment_message,sentiment_stars,review_clean,sentiment
0,Recebi bem antes do prazo estipulado.,3,recebi bem antes do prazo estipulado,NEUTRO
1,Parabéns lojas lannister adorei comprar pela I...,5,parabéns lojas lannister adorei comprar pela i...,POSITIVO


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33684 entries, 0 to 40949
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_comment_message  33684 non-null  object
 1   sentiment_stars         33684 non-null  int64 
 2   review_clean            33684 non-null  object
 3   sentiment               33684 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [32]:
df['sentiment'].value_counts(normalize=True)

,proportion
sentiment,
POSITIVO,0.496052
NEGATIVO,0.413817
NEUTRO,0.090132


La distribución de clases muestra un ligero predominio del sentimiento positivo (49.6 %), seguido muy de cerca por el negativo (41.4 %), mientras que el neutro es claramente minoritario (9.0 %).

Este patrón indica un conjunto altamente polarizado, donde la mayoría de los usuarios expresa opiniones claras (positivas o negativas) y pocos comentarios son realmente neutrales. Desde el punto de vista del modelado, el balance entre positivo y negativo es favorable y reduce el riesgo de sesgo fuerte hacia una sola clase; sin embargo, la clase neutra presenta un desbalance significativo, lo que puede dificultar su correcta identificación.

En consecuencia, es recomendable evaluar el desempeño con métricas por clase (especialmente recall y F1 para neutro) y considerar el uso de ponderación de clases durante el entrenamiento para evitar que esta categoría sea absorbida por las clases polarizadas.

## 6.-Exportando los DataSets a .csv para su uso en el entrenamiento del modelo.

In [34]:
df.to_csv('df_olist_pr.csv', index=False)